This notebook parses the benchmark results.

In [11]:
from pathlib import Path
from typing import NamedTuple
from itertools import product

import pandas as pd

## Constants

In [12]:
DATA_DIR = Path("../data/newresults/")
PROBLEMS = [
    "JSP",
    "FJSP",
    "NPFSP",
    "NW-PFSP",
    "HFSP",
    "PFSP",
    "SDST-PFSP",
    "TCT-PFSP",
    "TT-PFSP",
    "RCPSP",
    "MMRCPSP",
    "RCMPSP"
]
SOLVERS = ["ortools", "cpoptimizer"]
TIME_LIMITS = [120, 900]

## Helpers

In [13]:
class Result(NamedTuple):
    instance: str
    status: str
    objective: float
    lower_bound: float
    time: float

def parse_results(loc: Path) -> list[Result]:
    with open(loc, "r") as fh:
        text = fh.read()

    text = text[text.index("Instance"):] # start at results header
    lines = [line.strip() for line in text.split("\n")]
    lines = lines[2:] # skip header
    lines = iter(lines)

    results = []
    while (line := next(lines)): # continue until next empty line
        # try:
        instance, status, objective, lower_bound, time = line.split()
        result = Result(instance, status, float(objective), float(lower_bound), float(time))
        # except:
        #     instance, status, objective, time = line.split()
        #     result = Result(instance, status, float(objective), float(time))
        results.append(result)

    return results

## Parsing

In [14]:
class Experiment(NamedTuple):
    problem: str
    solver: str
    time_limit: int
    instance: str
    status: str
    objective: float
    lower_bound: float
    time: float

data = [] 
for problem, solver, time_limit in product(PROBLEMS, SOLVERS, TIME_LIMITS):
    try:
        loc = DATA_DIR / problem / solver / str(time_limit) / "results.txt"
        results = parse_results(loc)
        
        for result in results:
            data.append(Experiment(problem, solver, time_limit, *result))
    except Exception as e:
        print(loc, e)

In [15]:
df = pd.DataFrame(data)
df.head()

,problem,solver,time_limit,instance,status,objective,lower_bound,time
0,JSP,ortools,120,1.txt,Optimal,1231.0,1231.0,5.70
1,JSP,ortools,120,10.txt,Optimal,1241.0,1241.0,20.19
2,JSP,ortools,120,100.txt,Feasible,3846.0,3604.0,120.06
3,JSP,ortools,120,101.txt,Feasible,4581.0,4380.0,120.04
4,JSP,ortools,120,102.txt,Optimal,4725.0,4725.0,91.19


In [16]:
df.to_csv("../data/results.csv", index=False)